# problema di trasporto

Una società di logistica movimenta container vuoti da M magazzini a P porti

La richiesta dei porti (vedi tabella) può essere soddisfatta prelevando i container vuoti da uno qualsiasi dei magazzini. 

![tab1](./tab1.png)

Ogni magazzino, tuttavia, ha una disponibilità limitata di container (vedi tabella)

![tab2](./tab2.png)

La movimentazione avviene attraverso una flotta di camion, ognuno dei quali può trasportare al massimo 2 container. 

Il costo di ogni viaggio dipende dalla distanza che intercorre tra magazzino e porto ed è descritto dalla seguente tabella:

![tab3](./tab3.png)

Come devono essere organizzate le consegne al fine di minimizzare il costo totale di movimentazione?


## modello di programmazione matematica

![img1](./img1.png)

![img2](./img2.png)

In [50]:
from pyomo.environ import *
from pyomo.opt import SolverStatus, TerminationCondition

model = ConcreteModel()

nomi_magazzini = [ 'La Spezia', 'Trieste', 'Ancona', 'Napoli', 'Bari' ]
disp_magazzini = [ 20, 15, 25, 33, 21 ]
ind_magazzini = range(len(nomi_magazzini))
assert len(nomi_magazzini) == len(disp_magazzini)

nomi_porti = [ 'Padova', 'Arezzo', 'Roma', 'Teramo', 'Lecce', 'Catanzaro' ]
rich_porti = [ 10, 12, 20, 24, 18, 40 ]
ind_porti = range(len(nomi_porti))
assert len(nomi_porti) == len(rich_porti)

costi_list = [[8700, 3450, 10650, 21450, 24300],
          [11400, 10200, 4950, 11400, 18300], 
          [15150, 15900, 8550, 6600, 13500],
          [19650, 13500, 4650, 7200, 9450], 
          [30300, 25200, 16500, 9150, 2850], 
          [32160, 32910, 22410, 11160, 9990]]

costi = {}
for j in ind_porti:
    for i in ind_magazzini:
        costi[j, i] = costi_list[j][i]

model.i = RangeSet(0,len(nomi_magazzini))
model.j = RangeSet(0,len(nomi_porti))

model.m = Set(initialize=disp_magazzini)
model.r = Set(initialize=rich_porti)

model.c = Param(model.i, model.j, initialize=costi)

model.x = Var(model.j, model.i, domain=NonNegativeReals)
model.y = Var(model.j, model.i, domain=NonNegativeReals)

obj_expr = sum(sum(model.c[j, i]*model.y[j, i] for i in ind_magazzini) for j in ind_porti)

model.cost = Objective(expr = obj_expr, sense=minimize)

model.constraints = ConstraintList()

for i in ind_magazzini:
    model.constraints.add(expr = sum(model.x[j, i] for j in ind_porti) <= disp_magazzini[i])

for j in ind_porti:
    model.constraints.add(expr = sum(model.x[j, i] for i in ind_magazzini) >= rich_porti[j])

for j in ind_porti:
    for i in ind_magazzini:
        model.constraints.add(expr = 2*model.y[j, i] >= model.x[j, i])

result = SolverFactory('glpk').solve(model)

print(result)



Problem: 
- Name: unknown
  Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 42
  Number of variables: 61
  Number of nonzeros: 121
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: other
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.008928775787353516

